In [1]:
import pandas as pd

In [11]:
# !pip3 install -U sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 9.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 8.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 10.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1

In [20]:
# !pip3 install faiss-cpu

  Using cached faiss_cpu-1.7.4-cp311-cp311-macosx_10_9_x86_64.whl (6.5 MB)


In [21]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

/Users/goodyoung/Desktop/대학교/gytest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("서울시 공공데이터 최종.csv")

In [8]:
df['서비스명']

0                          서울시 강동구 명일2동 주민센터 새소식 정보
1                          서울시 강동구 암사2동 주민센터 새소식 정보
2                            서울시 강동구 길동 주민센터 새소식 정보
3                              서울시 성동구 통신판매업 인허가 정보
4                            서울시 성북구 건물위생관리업 인허가 정보
                           ...                     
7393         서울시 연평균최고기온 2005년 위치정보 (좌표계: ITRF2000)
7394         서울시 연평균최저기온 2008년 위치정보 (좌표계: ITRF2000)
7395    서울시 자치구별 TOE 환산량 2008년 위치정보 (좌표계: ITRF2000)
7396     서울시 행정동별 전력 사용량 2008년 위치정보 (좌표계: ITRF2000)
7397                          서울시 사회적기업 지원 기관 목록 정보
Name: 서비스명, Length: 7398, dtype: object

In [6]:
df

,서비스ID,서비스명,대분류,중분류,소분류,제공기관,제공부서명,시스템명,담당자명,담당자연락처,갱신주기,최종갱신일자,제공사이트,제공형식,서비스설명,urls
0,OA-12615,서울시 강동구 명일2동 주민센터 새소식 정보,공공데이터,자치구 및 자치구산하,일반행정,강동구,강동구 명일2동,강동구 홈페이지,강동구 대표전화,02-1577-1188,일간,NaN,강동구 홈페이지(https://www.gangdong.go.kr/web/dongre...,"Sheet,Api","명일2동 주민센터에서 주민에게 알리는 소식 정보로서 게시물ID, 게시물순번, 게시물...",https://data.seoul.go.kr/dataList/OA-12615/S/1...
1,OA-12621,서울시 강동구 암사2동 주민센터 새소식 정보,공공데이터,자치구 및 자치구산하,일반행정,강동구,강동구 암사2동,강동구 홈페이지,강동구 대표전화,02-1577-1188,일간,NaN,강동구 홈페이지(https://www.gangdong.go.kr/web/dongre...,"Sheet,Api","암사2동 주민센터에서 주민에게 알리는 소식 정보로서 게시물ID, 게시물순번, 게시물...",https://data.seoul.go.kr/dataList/OA-12621/S/1...
2,OA-12611,서울시 강동구 길동 주민센터 새소식 정보,공공데이터,자치구 및 자치구산하,일반행정,강동구,강동구 길동,강동구 홈페이지,강동구 대표전화,02-1577-1188,일간,NaN,강동구 홈페이지(https://www.gangdong.go.kr/web/dongre...,"Sheet,Api","길동 주민센터에서 주민에게 알리는 소식 정보로서 게시물ID, 게시물순번, 게시물제목...",https://data.seoul.go.kr/dataList/OA-12611/S/1...
3,OA-18805,서울시 성동구 통신판매업 인허가 정보,공공데이터,서울시(본청),산업/경제,성동구,스마트도시정책관 빅데이터담당관,지방행정 인허가 데이터개방,성동구 대표전화,02-2286-5114,수시,2023-04-14,지방행정 인허가 데이터개방(http://localdata.kr),"Sheet,Api","성동구의 TV홈쇼핑, 인터넷, 카탈로그, 신문잡지, 기타의 방법으로 가구, 가전, ...",https://data.seoul.go.kr/dataList/OA-18805/S/1...
4,OA-19284,서울시 성북구 건물위생관리업 인허가 정보,공공데이터,서울시(본청),환경,성북구,스마트도시정책관 빅데이터담당관,지방행정 인허가 데이터개방,성북구 대표전화,02-2241-3114,수시,2023-04-14,지방행정 인허가 데이터개방(http://localdata.kr),"Sheet,Api",성북구의 공중이 이용하는 건축물·시설물등의 청결유지와 실내공기정화를 위한 청소 등을...,https://data.seoul.go.kr/dataList/OA-19284/S/1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7393,OA-350,서울시 연평균최고기온 2005년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),환경,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Api,Sheet,Chart","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-350/S/1/d...
7394,OA-357,서울시 연평균최저기온 2008년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),환경,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Sheet,Api,Chart","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-357/S/1/d...
7395,OA-342,서울시 자치구별 TOE 환산량 2008년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),산업/경제,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Sheet,Api,Chart","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-342/S/1/d...
7396,OA-330,서울시 행정동별 전력 사용량 2008년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),산업/경제,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Sheet,Api","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-330/S/1/d...


In [25]:
test_list = df.서비스명.to_list()

In [22]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

NameError: name 'data' is not defined

In [26]:
encoded_data = model.encode(test_list)
print('임베딩 된 벡터 수 :', len(encoded_data))

임베딩 된 벡터 수 : 7398


In [32]:
gg = pd.DataFrame(encoded_data)

In [33]:
# gg.to_csv("임베딩.csv",index = False)

In [34]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(3))

In [45]:
encoded_data

array([[-1.5167751e+00, -4.1165814e-01,  5.4186779e-01, ...,
         2.1649906e-01,  4.4315782e-01, -5.9748107e-01],
       [-1.5166954e+00, -3.6464989e-01,  4.8307404e-01, ...,
         1.8366843e-01,  4.4456688e-01, -6.2987757e-01],
       [-1.5393742e+00, -3.9079607e-01,  5.3113961e-01, ...,
         2.4665739e-01,  4.2285979e-01, -5.8793980e-01],
       ...,
       [-1.4407890e+00, -6.7301702e-01,  8.2360804e-01, ...,
         9.2588395e-02,  3.9122321e-04, -6.4048630e-01],
       [-1.4337369e+00, -6.2277317e-01,  7.9596609e-01, ...,
         1.0572040e-01,  4.7506336e-02, -7.6883447e-01],
       [-1.4101390e+00, -3.5021406e-01,  6.7807555e-01, ...,
         1.4625199e-01,  3.4862927e-01, -7.4407798e-01]], dtype=float32)

In [37]:
index.add_with_ids(encoded_data, np.array(range(0, len(test_list))))

In [38]:
index

<faiss.swigfaiss_avx2.IndexIDMap; proxy of <Swig Object of type 'faiss::IndexIDMapTemplate< faiss::Index > *' at 0x1539ec240> >

In [39]:
faiss.write_index(index, 'abc_news')

In [42]:
def search(query):
    t = time.time()
    query_vector = model.encode([query])
    k = 5
    top_k = index.search(query_vector, k)
    print('total time: {}'.format(time.time() - t))
    return [test_list[_id] for _id in top_k[1].tolist()[0]]

In [44]:
query = str(input())
results = search(query)

print('results :')
for result in results:
    print('\t', result)

 하천


total time: 0.06947875022888184
results :
	 한울타리 웹진
	 사회적거리두기로 인한 어려움
	 보유 자산
	 가구원 최종 학교
	 서울시 하천기본계획정보


In [47]:
df['서비스ID']

0       OA-12615
1       OA-12621
2       OA-12611
3       OA-18805
4       OA-19284
          ...   
7393      OA-350
7394      OA-357
7395      OA-342
7396      OA-330
7397     OA-2532
Name: 서비스ID, Length: 7398, dtype: object

In [ ]:
#